# Clone repository

In [ ]:
# !git clone https://github.com/CjangCjengh/vits.git
!git clone https://huggingface.co/spaces/innnky/soft-vits-vc


In [1]:
#win
%cd F:\shixi\CS\vits4\soft-vits-vc
#colab
#%cd /content/vits1
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
%pip install -r requirements.txt
!sudo apt-get install espeak -y

f:\shixi\CS\vits4\soft-vits-vc
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://download.pytorch.org/whl/cu113
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unpack dataset

In [ ]:
!sudo apt-get install p7zip-full
!7z x ../drive/MyDrive/dataset.zip

# Alignment

In [3]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

f:\shixi\CS\vits4\soft-vits-vc\monotonic_align
running build_ext
copying build\lib.win-amd64-3.7\monotonic_align\core.pyd -> monotonic_align
f:\shixi\CS\vits4\soft-vits-vc


# Clean

In [186]:
!python preprocess.py --text_index 1 --text_cleaners japanese_cleaners2 --filelists filelists/shioriko_train_filelist.txt filelists/shioriko_val_filelist.txt filelists/shioriko_test_filelist.txt

START: filelists/shioriko_train_filelist.txt
START: filelists/shioriko_val_filelist.txt
START: filelists/shioriko_test_filelist.txt


# Train

In [13]:
!python train.py -c configs/shioriko_base.json -m shioriko_base

^C


In [1]:
import os
os.system('cd monotonic_align && python setup.py build_ext --inplace && cd ..')

import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

import librosa
import torch

import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write
from scipy.io.wavfile import read


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    print(text_norm.shape)
    return text_norm


hps = utils.get_hparams_from_file("configs/shioriko_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
import numpy as np

hubert = torch.hub.load("bshall/hubert:main", "hubert_soft")


def vc_fn(input_audio):
    if input_audio is None:
        return "You need to upload an audio", None
    
    sampling_rate, audio_path = input_audio
    sampling_rate, audio = read(audio_path)
    # print(audio.shape,sampling_rate)
    duration = audio.shape[0] / sampling_rate
    if duration > 30:
        return "Error: Audio is too long", None
    audio = (audio / np.iinfo(audio.dtype).max).astype(np.float32)
    if len(audio.shape) > 1:
        audio = librosa.to_mono(audio.transpose(1, 0))
    if sampling_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sampling_rate, target_sr=16000)
    source = torch.FloatTensor(audio).unsqueeze(0).unsqueeze(0)
    print(source.shape)
    with torch.inference_mode():
        units = hubert.units(source)

    stn_tst = torch.FloatTensor(units.squeeze(0))
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=0.1, noise_scale_w=0.1, length_scale=1)[0][
            0, 0].data.float().numpy()

    return "Success", (hps.data.sampling_rate, audio)

Using cache found in C:\Users\15800/.cache\torch\hub\bshall_hubert_main


In [14]:
_ = utils.load_checkpoint("G_88000.pth", net_g, None)
# _ = utils.load_checkpoint("../drive/MyDrive/shioriko_base/G_34000.pth", net_g, None)
#_ = utils.load_checkpoint("/content/drive/MyDrive/japanese_base2_shioriko/G_94000.pth", net_g, None)


In [16]:
_, data = vc_fn((22050, "F:/素材/栞子wav/丁真.wav"))
sampling_rate, audio = data
write("example1.wav", 22050, audio)

d:\ProgramData\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:48: WavFileWarning: Chunk (non-data) not understood, skipping it.
d:\ProgramData\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:48: WavFileWarning: Incomplete chunk ID: b'\x00', ignoring it.


torch.Size([1, 1, 474880])
